<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/process/valvesCvCg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Valve simulation with NeqSim
This notebook demonstrates how control valves are simulated using the NeqSim process library. It shows how the valve flow coefficients **Cv** and **Cg** are handled, how to size a valve, and how to control downstream pressure or flow by setting the valve opening.

In [ ]:
%%capture
!pip install neqsim
import neqsim
from neqsim.thermo.thermoTools import *
from neqsim.process import clearProcess, stream, valve, runProcess


## Basic valve simulation
Create a simple gas stream and let it flow through a valve. The downstream pressure is fixed using `setOutletPressure`. When a `Cv` value is specified the mass flow through the valve is calculated from the pressure drop.

In [ ]:
inletTemperature = 20.0  # C
inletPressure = 100.0  # bara
outletPressure = 50.0  # bara

fluid1 = fluid('srk')
fluid1.addComponent('methane', 1.0)
fluid1.setTemperature(inletTemperature, 'C')
fluid1.setPressure(inletPressure, 'bara')
fluid1.setTotalFlowRate(1000.0, 'kg/hr')

clearProcess()
stream1 = stream(fluid1)
valve1 = valve(stream1)
valve1.setOutletPressure(outletPressure, 'bara')
valve1.setCv(80.0)
runProcess()

print('Valve outlet pressure', valve1.getOutStream().getPressure('bara'), 'bara')
print('Valve outlet temperature', valve1.getOutStream().getTemperature('C'), 'C')
print('Valve mass flow', valve1.getOutStream().getFlowRate('kg/hr'))
print('Valve Cv used', valve1.getCv())

### Cv and Cg
The valve can alternatively be characterised with the gas sizing coefficient **Cg**. In NeqSim the relationship between Cv and Cg follows standard valve sizing correlations. Setting one automatically updates the other.

In [ ]:
valve1.setCg(95.0)  # set gas sizing coefficient
print('Updated Cv from Cg:', valve1.getCv())
print('Cg now:', valve1.getCg())

## Valve sizing
The required size of a valve is normally given by its flow coefficient. For liquid service the metric value is Kv while for gas service the coefficients Cv or Cg are used. Kv is related to Cv by `Kv = 0.865 * Cv`. The table below gives typical Cv values for different valve types.

In [ ]:
Kv = stream1.getFlowRate('m3/hr')/math.sqrt(inletPressure-outletPressure)
Cv = Kv/0.865
print('Calculated Kv', Kv)
print('Corresponding Cv', Cv)

## Controlling downstream pressure by valve opening
Instead of fixing the outlet pressure it is possible to set a valve opening and let NeqSim calculate the resulting flow or downstream pressure. The method `setPercentValveOpening` defines the degree of opening from 0 to 100%.

In [ ]:
valve1.setOutletPressure(None)  # clear fixed pressure
valve1.setPercentValveOpening(50.0)
runProcess()
print('Downstream pressure with 50% opening', valve1.getOutStream().getPressure('bara'))
print('Resulting flow', valve1.getOutStream().getFlowRate('kg/hr'))